#### Eager Execution

TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, without building graphs: operations return concrete values instead of constructing a computational graph to run later. This makes it easy to get started with TensorFlow and debug models, and it reduces boilerplate as well. To follow along with this guide, run the code samples below in an interactive python interpreter.

Eager execution supports most TensorFlow operations and GPU acceleration.

Let’s have a quick look at the 🤗 Transformers library features. The library downloads pretrained models for Natural Language Understanding (NLU) tasks, such as analyzing the sentiment of a text, and Natural Language Generation (NLG), such as completing a prompt with new text or translating in another language.

First we will see how to easily leverage the pipeline API to quickly use those pretrained models at inference. Then, we will dig a little bit more and see how the library gives you access to those models and helps you preprocess your data.

In [1]:
pip install transformers

     |████████████████████████████████| 1.9MB 21.8MB/s 
     |████████████████████████████████| 890kB 49.3MB/s 
     |████████████████████████████████| 3.2MB 52.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=4f5c519fc9adac4f06eddb01c08ee036887d1ea99a9b6c80f75baaa50b8c2bfe
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

In [3]:
classifier('i dont kmow how i feel')

[{'label': 'NEGATIVE', 'score': 0.9682340025901794}]

In [20]:
#url='https://raw.githubusercontent.com/drainganggtb/FinalProject/main/clean_tweets.csv'
url='https://raw.githubusercontent.com/drainganggtb/FinalProject/main/new_df.csv'

In [30]:
# load data
import pandas as pd
statement_df = pd.read_csv(url,encoding='UTF-8',header=None,sep=',',error_bad_lines=False,usecols=[3],lineterminator='\n').drop(0,0)
statement_df.reset_index(drop=True, inplace=True)
statement_df.columns = ['statement']

In [25]:
statement_df

,statement
0,Machine learning on distributed Dask using Ama...
1,RT @realmleviticus: Top 4 python programming l...
2,RT @Eli_Krumova: Classification with Localizat...
3,RT @axelrod_eric: Artificial intelligence and ...
4,RT @Omkar_Raii: #AgriTech startups like @AgNex...
...,...
20026,RT @intuitibits: Airtool 2.2 is out. Adds supp...
20027,#XSLLabs #XSL #SYL #Sylare
20028,"RT @CZDS: PGA Tour signs up with AWS, looks to..."
20029,RT @Strat_AI: Build a #Chatbot with #DeepLearn...


In [31]:
statement_df = statement_df.dropna()
statement_df

,statement
0,Machine learning on distributed Dask using Ama...
1,RT @realmleviticus: Top 4 python programming l...
2,RT @Eli_Krumova: Classification with Localizat...
3,RT @axelrod_eric: Artificial intelligence and ...
4,RT @Omkar_Raii: #AgriTech startups like @AgNex...
...,...
20026,RT @intuitibits: Airtool 2.2 is out. Adds supp...
20027,#XSLLabs #XSL #SYL #Sylare
20028,"RT @CZDS: PGA Tour signs up with AWS, looks to..."
20029,RT @Strat_AI: Build a #Chatbot with #DeepLearn...


In [32]:
tweets_list = statement_df['statement'].tolist()

In [33]:
result=[]
for data in range(len(tweets_list)):
  result.append(classifier(tweets_list[data]))
  

In [34]:
label=[]
score=[]
for d in result:
  for e in d:
    label.append(e['label'])
    score.append(e['score'])
 

In [39]:
df = pd.DataFrame(label,columns=['label'])
df

,label
0,NEGATIVE
1,POSITIVE
2,NEGATIVE
3,NEGATIVE
4,NEGATIVE
...,...
20026,NEGATIVE
20027,NEGATIVE
20028,NEGATIVE
20029,NEGATIVE


In [40]:
df.value_counts(normalize=True) * 100

label   
NEGATIVE    74.953822
POSITIVE    25.046178
dtype: float64